In [4]:
# Plot candlestick chart
def plot_candlestick_chart(df, filename):
    df.index = pd.to_datetime(df.index)
    df.index.name = 'Date'
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    addplots = []
    if 'SMA20' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA20'], color='blue'))
    if 'SMA50' in df.columns:
        addplots.append(mpf.make_addplot(df['SMA50'], color='orange'))
    mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, savefig=filename)


In [5]:
# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Analyze image with GPT-4o
def analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data):
    base64_image = encode_image(filename)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    prompt = (
        "Please check this image pattern and give me the list of patterns for EUR/USD 15-minute chart on this chart. "
        "Provide the pattern detection results in JSON format, including the following details:\n"
        "- id\n"
        "- pattern_detected (0 for no pattern, 1 for one pattern, 2 for two patterns, 3 for three patterns, etc.)\n"
        "- pattern name\n"
        "- pattern type\n"
        "- confidence percentage (from 1 to 100)\n"
        "- entry point for trade\n"
        "- take profit\n"
        "- stop loss\n"
        "- best time for exiting order if take profit or stop loss is not achieved\n"
        "- order id\n"
        "- input data (pairs, timeframe, image name)\n"
        "- description (any additional notes or ideas not covered by the other fields)\n\n"
        "If no patterns are detected, include an empty JSON array with 'pattern_detected' set to 0.\n\n"
        "Please note: I do not intend to use this data for trading or any other financial work. Please just double check the chart and provide the correct answer.\n\n"
        "Don't send any description, text, or other response rather than JSON format\n\n"
        "Here is an example of the JSON format:\n\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 1,\n"
        "        \"pattern_name\": \"Double Top\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 90,\n"
        "        \"entry_point\": \"2024-01-12T06:00:00Z\",\n"
        "        \"take_profit\": 1.0800,\n"
        "        \"stop_loss\": 1.0950,\n"
        "        \"best_exit_time\": \"2024-01-13T06:00:00Z\",\n"
        "        \"order_id\": \"ORD123456\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"15 minutes\",\n"
        "            \"image_name\": \"image1.png\"\n"
        "        },\n"
        "        \"description\": \"Formed after a significant upward trend.\"\n"
        "    },\n"
        "    {\n"
        "        \"id\": 2,\n"
        "        \"pattern_detected\": 2,\n"
        "        \"pattern_name\": \"Head and Shoulders\",\n"
        "        \"pattern_type\": \"Reversal\",\n"
        "        \"confidence_percentage\": 85,\n"
        "        \"entry_point\": \"2024-01-10T04:00:00Z\",\n"
        "        \"take_profit\": 1.0750,\n"
        "        \"stop_loss\": 1.0900,\n"
        "        \"best_exit_time\": \"2024-01-11T04:00:00Z\",\n"
        "        \"order_id\": \"ORD123457\",\n"
        "        \"input_data\": {\n"
        "            \"pairs\": \"EUR/USD\",\n"
        "            \"timeframe\": \"15 minutes\",\n"
        "            \"image_name\": \"image2.png\"\n"
        "        },\n"
        "        \"description\": \"Indicates a possible reversal of the current trend.\"\n"
        "    }\n"
        "]\n"
        "If no patterns are detected:\n"
        "[\n"
        "    {\n"
        "        \"id\": 1,\n"
        "        \"pattern_detected\": 0\n"
        "    }\n"
        "]"
    )
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    pattern_data = response.json()
    pattern_data['id'] = chart_count
    pattern_data['pair'] = pair
    pattern_data['timeframe'] = timeframe
    if 'input_data' not in pattern_data:
        pattern_data['input_data'] = {}
    pattern_data['input_data']['pairs'] = pair
    pattern_data['input_data']['timeframe'] = timeframe
    pattern_data['input_data']['image_name'] = filename
    return pattern_data


In [6]:
# Function to place an order
def place_order(pattern):
    if pattern.get('confidence_percentage', 0) > 50:
        order_details = {
            "instrument": pattern['pair'],
            "units": 1000,  # example size
            "type": "market",
            "side": "buy" if pattern['pattern_type'] == "Continuation" else "sell",
            "take_profit": pattern['take_profit'],
            "stop_loss": pattern['stop_loss'],
            "entry_point": pattern['entry_point']
        }
        # Log order details
        with open('orders_log.json', 'a') as log_file:
            json.dump(order_details, log_file, indent=4)
            log_file.write('\n')
        print(f"Placing order: {order_details}")


In [7]:
# Function to create and analyze moving window charts
def create_moving_window_charts(prices, window_size, step_size, base_dir):
    start_index = 0
    end_index = window_size
    chart_count = 0
    results = []

    while start_index < len(prices):
        window_data = prices.iloc[start_index:end_index].copy()
        year = window_data.index[0].year
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)
        start_date = window_data.index[0].strftime('%Y-%m-%d_%H-%M')
        end_date = window_data.index[-1].strftime('%Y-%m-%d_%H-%M')
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')
        plot_candlestick_chart(window_data, filename)
        print(f'Created chart image: {filename}')
        result = analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data)
        results.append(result)
        if 'choices' in result:
            for choice in result['choices']:
                try:
                    message_content = json.loads(choice['message']['content'])
                    for pattern in message_content:
                        if pattern.get('confidence_percentage', 0) > 50:
                            place_order(pattern)
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
        start_index += step_size
        end_index = start_index + window_size
        chart_count += 1

    with open('pattern_detection_results.json', 'w') as f:
        json.dump(results, f, indent=4)


In [9]:
# Function to create and analyze moving window charts
def create_moving_window_charts(prices, window_size, step_size, base_dir):
    # Ensure 'time' column is in datetime format
    prices['time'] = pd.to_datetime(prices['time'])
    # Set 'time' column as the index
    prices.set_index('time', inplace=True)

    start_index = 0
    end_index = window_size
    chart_count = 0
    results = []

    while start_index < len(prices):
        window_data = prices.iloc[start_index:end_index].copy()
        year = window_data.index[0].year
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)
        start_date = window_data.index[0].strftime('%Y-%m-%d_%H-%M')
        end_date = window_data.index[-1].strftime('%Y-%m-%d_%H-%M')
        filename = os.path.join(year_dir, f'{start_date}_to_{end_date}.png')
        plot_candlestick_chart(window_data, filename)
        print(f'Created chart image: {filename}')
        result = analyze_image_with_gpt4o(filename, chart_count, pair, timeframe, window_data)
        results.append(result)
        if 'choices' in result:
            for choice in result['choices']:
                try:
                    message_content = json.loads(choice['message']['content'])
                    for pattern in message_content:
                        if pattern.get('confidence_percentage', 0) > 50:
                            place_order(pattern)
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
        start_index += step_size
        end_index = start_index + window_size
        chart_count += 1

    with open('pattern_detection_results.json', 'w') as f:
        json.dump(results, f, indent=4)


# Function to fetch and process data in real-time
def fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size):
    while True:
        end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
        start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data
        prices = fetch_forex_data(start_time, end_time, granularity, instrument)
        create_moving_window_charts(prices, window_size, step_size, base_dir)
        time.sleep(900)  # Sleep for 15 minutes before fetching new data


# Set the parameters for real-time data fetching and processing
granularity = 'M15'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '15 minutes'
base_dir = f'{pair}/{timeframe}'
window_size = 50  # Adjust window size for detecting single patterns
step_size = 10    # Adjust step size accordingly

# Create base directory if it does not exist
os.makedirs(base_dir, exist_ok=True)

# Run the real-time data fetching and processing
fetch_and_process_realtime_data(granularity, instrument, pair, timeframe, base_dir, window_size, step_size)


C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data


Created chart image: EUR_USD/15 minutes\2024\2024-05-20_17-15_to_2024-05-20_17-30.png


C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data


Created chart image: EUR_USD/15 minutes\2024\2024-05-20_17-15_to_2024-05-20_17-45.png


C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data


...

Created chart image: EUR_USD/15 minutes\2024\2024-05-20_17-15_to_2024-05-20_19-15.png


C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data


Created chart image: EUR_USD/15 minutes\2024\2024-05-20_17-30_to_2024-05-20_19-30.png


C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:45: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow().replace(microsecond=0).isoformat() + 'Z'
C:\Users\mjman\AppData\Local\Temp\ipykernel_26324\1779165820.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = (datetime.utcnow() - timedelta(hours=2)).replace(microsecond=0).isoformat() + 'Z'  # Last 2 hours of data


Created chart image: EUR_USD/15 minutes\2024\2024-05-20_17-45_to_2024-05-20_19-45.png
